In [1]:
import numpy as np       # For numerical computations
import pandas as pd      # For data manipulation and analysis
import statsmodels.api as sm  # For econometric and statistical modeling
import scipy.stats as stats   # For statistical methods (e.g., Delta method)
import matplotlib.pyplot as plt  # For plotting (if needed for visualizations)
from tqdm import tqdm

In [2]:
# read file into dataframe
f = "nls_2008.txt"
columns = ['luwe', 'educ', 'exper', 'age', 'fed', 'med', 'kww', 'iq', 'white']
df = pd.read_csv(f, sep='\t', header=None, names=columns)
#print(df.head(10))

In [3]:
print("######### Problem 1 #########")
summary_stats = df.describe().loc[['min', 'max', 'mean', 'std']]
print("\n",summary_stats)

######### Problem 1 #########

           luwe       educ      exper        age        fed        med  \
min   4.055853   9.000000   5.000000  28.000000   0.000000   0.000000   
max   7.601318  18.000000  23.000000  38.000000  18.000000  18.000000   
mean  5.945158  13.470968  13.623656  33.094624   8.122581   9.816129   
std   0.442819   2.199353   3.827873   3.104161   5.072353   4.005588   

            kww          iq     white  
min   12.000000   50.000000  0.000000  
max   56.000000  145.000000  1.000000  
mean  35.787097  101.317204  0.873118  
std    7.631576   15.050951  0.333020  


In [4]:
print("######### Problem 2 #########")

# add column for experience-squared
df['exper_squared'] = df['exper'] ** 2

# define variables and fit model
X = sm.add_constant(df[['educ', 'exper', 'exper_squared']])
Y = df['luwe']
model = sm.OLS(Y, X).fit()
#print(model.summary())

# fit robust model
robust_model = sm.OLS(Y, X).fit(cov_type='HC0')

######### Problem 2 #########


In [5]:
# conventional model
print("\nConventional Estimates:")
print(model.params)
print("\nConventional Standard Errors:")
print(model.bse)

# robust model
print("\nRobust Estimates:")
print(robust_model.params)
print("\nRobust Standard Errors:")
print(robust_model.bse)


Conventional Estimates:
const            4.030551
educ             0.092514
exper            0.076777
exper_squared   -0.001886
dtype: float64

Conventional Standard Errors:
const            0.223018
educ             0.007581
exper            0.025003
exper_squared    0.000872
dtype: float64

Robust Estimates:
const            4.030551
educ             0.092514
exper            0.076777
exper_squared   -0.001886
dtype: float64

Robust Standard Errors:
const            0.213784
educ             0.007672
exper            0.024257
exper_squared    0.000851
dtype: float64


In [6]:
# variance-covariance matrices
print("Conventional Variance-Covariance Matrix")
print(model.cov_params())
print("\nRobust Variance-Covariance Matrix")
print(robust_model.cov_params())

Conventional Variance-Covariance Matrix
                  const          educ     exper  exper_squared
const          0.049737 -1.137084e-03 -0.004684   1.476767e-04
educ          -0.001137  5.746865e-05  0.000035  -5.506074e-07
exper         -0.004684  3.473218e-05  0.000625  -2.147994e-05
exper_squared  0.000148 -5.506074e-07 -0.000021   7.609628e-07

Robust Variance-Covariance Matrix
                  const          educ     exper  exper_squared
const          0.045704 -1.069321e-03 -0.004226   1.319140e-04
educ          -0.001069  5.885960e-05  0.000021  -2.960741e-08
exper         -0.004226  2.081008e-05  0.000588  -2.031156e-05
exper_squared  0.000132 -2.960741e-08 -0.000020   7.244931e-07


In [7]:
# calculate residual variance
resid_var = np.var(model.resid, ddof=1)
robust_resid_var = np.var(robust_model.resid, ddof=1)
print("Conventional Residual Variance:", resid_var)
print("Conventional Residual Variance:", robust_resid_var)

Conventional Residual Variance: 0.16807722965494284
Conventional Residual Variance: 0.16807722965494284


In [8]:
print("######### Problem 3 #########")

# new dataframe with adjusted data for -1 education level
df_educminus1 = df.copy()
df_educminus1['educ'] = df_educminus1['educ'] - 1
df_educminus1['exper'] = df_educminus1['age'] - df_educminus1['educ'] - 6
df_educminus1['exper_squared'] = df_educminus1['exper'] ** 2

# predict using problem 2 model
X_educminus1 = sm.add_constant(df_educminus1[['educ', 'exper', 'exper_squared']])
predicted_luwe_educminus1 = model.predict(X_educminus1)
#print(predicted_luwe_educminus1)

# calculate effect
avg_luwe = df['luwe'].mean()
avg_luwe_educminus1 = predicted_luwe_educminus1.mean()
change_in_log_wages = avg_luwe_educminus1 - avg_luwe
print("Effect on average log weekly wages of decreasing education by one year:", change_in_log_wages)

######### Problem 3 #########
Effect on average log weekly wages of decreasing education by one year: -0.0690070952327817


In [9]:
print("######### Problem 4 #########")

# new covariates
df['new_educ'] = df['educ'] - 1
df['new_exper'] = df['exper'] + 1
df['new_exper_squared'] = df['new_exper'] ** 2

X_redefined = sm.add_constant(df[['new_educ', 'new_exper', 'new_exper_squared']])
model_redefined = sm.OLS(df['luwe'], X_redefined).fit()
#print(model_redefined.summary())
print("Effect of decreasing education by one year (new_educ coefficient):", model_redefined.params['new_educ'])

######### Problem 4 #########
Effect of decreasing education by one year (new_educ coefficient): 0.09251371046597111


In [10]:
print("The effect can be found but the result will be different.")

The effect can be found but the result will be different.


In [11]:
print("######### Problem 5 #########")

# dataframe to reflect policy change
df_policy = df.copy()
df_policy['educ'] = df_policy['educ'].apply(lambda x: 12 if x < 12 else x)
df_policy['exper'] = df_policy['age'] - df_policy['educ'] - 6
df_policy['exper_squared'] = df_policy['exper'] ** 2

# predict using problem 2 model
X_policy = sm.add_constant(df_policy[['educ', 'exper', 'exper_squared']])
predicted_luwe_policy = model.predict(X_policy)

# calculate effect
original_avg_luwe = df['luwe'].mean()
policy_avg_luwe = predicted_luwe_policy.mean()
policy_effect = policy_avg_luwe - original_avg_luwe
print("Effect on average log weekly wages after increasing education to 12 years:", policy_effect)

######### Problem 5 #########
Effect on average log weekly wages after increasing education to 12 years: 0.012329600532393847


In [12]:
print("######### Problem 6 #########")

# delta method
gradient = X_policy.mean().values  # mean of the covariates represents the average change
var_conventional = np.dot(np.dot(gradient.T, model.cov_params()), gradient)
var_robust = np.dot(np.dot(gradient.T, robust_model.cov_params()), gradient)

# calculate the standard errors
se_conventional = np.sqrt(var_conventional)
se_robust = np.sqrt(var_robust)
print("Standard error (conventional):", se_conventional)
print("Standard error (robust):", se_robust)

######### Problem 6 #########
Standard error (conventional): 0.013527813507049036
Standard error (robust): 0.013516957752480786


In [13]:
print("######### Problem 7 #########")

def bootstrap_effect(data, data_policy, model_conventional, model_robust):
    # resample data with replacement
    indices = np.random.choice(data.index, size=len(data), replace=True)
    sample = data.loc[indices]
    sample_policy = data_policy.loc[indices]
    
    # predict w/ conventional model
    predicted_luwe_policy_conventional = model.predict(sm.add_constant(sample_policy[['educ', 'exper', 'exper_squared']])).mean()
    original_luwe_mean_conventional = sample['luwe'].mean()
    effect_conventional = predicted_luwe_policy_conventional - original_luwe_mean_conventional
    
    # predict w/ robust model
    predicted_luwe_policy_robust = model_robust.predict(sm.add_constant(sample_policy[['educ', 'exper', 'exper_squared']])).mean()
    original_luwe_mean_robust = sample['luwe'].mean()
    effect_robust = predicted_luwe_policy_robust - original_luwe_mean_robust
    
    return effect_conventional, effect_robust

# run bootstraps
n_bootstraps = 100000
bootstrap_results = [bootstrap_effect(df, df_policy, model, robust_model) for _ in tqdm(range(n_bootstraps))]
bootstrap_conventional = [result[0] for result in bootstrap_results]
bootstrap_robust = [result[1] for result in bootstrap_results]

# standard error is calculated as the standard deviation of the bootstrap estimates
bootstrap_se_conventional = np.std(bootstrap_conventional)
bootstrap_se_robust = np.std(bootstrap_robust)
print("Bootstrap standard error (conventional):", bootstrap_se_conventional)
print("Bootstrap standard error (robust):", bootstrap_se_robust)

######### Problem 7 #########


100%|██████████████████████████████████████████████████████████████████| 100000/100000 [06:39<00:00, 250.48it/s]

Bootstrap standard error (conventional): 0.013485691151961146
Bootstrap standard error (robust): 0.013485691151961146


In [14]:
print("The conventional and robust bootstrap standard errors are very similar to the analytic standard errors.")

The conventional and robust bootstrap standard errors are very similar to the analytic standard errors.
